## Code to geocode data from EXCEL, create a GeoJSON file, and plot a simple map


In the first step, we have to install and import the necessary Python packages. Pandas is needed for working with tabular data, folium is needed for mapping, geopy for geocoding, and json for the generation of a valid GeoJSON file. Time is needed to introduce a delay when calling the geocoding API. Otherwise we might hit a timeout.


In [ ]:
# Install required packages
!pip install --quiet pandas openpyxl folium geopy

import pandas as pd
import folium
from geopy.geocoders import Nominatim
import json
import time

In the next step, the script will prompt you to upload an EXCEL file from your local machine. The file can have as many data columns as you like but must include one column called "Address" with some spatial information. My own test file has three columns: event, date, and address. The address columns in my sample contains the names of different places in England.


In [ ]:
# Upload Excel file
from google.colab import files
uploaded = files.upload()

We now read the file and create a so-called dataframe, a two-dimensional data structure, that we can display as a preview.


In [ ]:
# Load the uploaded Excel file and display if valid

if 'Address' not in df.columns:
    raise ValueError("The Excel file must have a column named 'Address'. Please re-run the code!")
else:
    excel_file = list(uploaded.keys())[0]
    df = pd.read_excel(excel_file)
    display(df)

The following step is the actual geocoding. This means that the place name found in the "Address" column of your file will be connected with a specific latitude and longitude. These coordinates will come from Open Street Maps. Empty cells will be ignored. Please note that my approach below is rather simplistic. A more Pythonic solution would be to use lambda or apply for iterating over all values in the "Address" column, but in my test runs with different data, I found that this would also require more error handling for possible key errors.


In [ ]:
# Start Nominatim geolocator
geolocator = Nominatim(user_agent="geojson_mapper", timeout=10)

# Test geocoding functionality with one place

location_1 = geolocator.geocode("Oxford")
print("These are the OSM coordinates: ", location_1.latitude, location_1.longitude)

# Make list for new latitude values

coord_lat = []

# Make list for new longitude values

coord_lon = []

# Geocode places and add values to lists

def geocode_address(address):
    try:
        location = geolocator.geocode(address, exactly_one=True)
        if location:
            coord_lat.append(location.latitude)
            coord_lon.append(location.longitude)
        else:
            coord_lat.append(None)
            coord_lon.append(None)
    except Exception as e:
        print(f"An error occurred with address '{address}': {e}")
        coord_lat.append(None)
        coord_lon.append(None)

# Clean address column and geocode
df['Address'] = df['Address'].astype(str)
for address in df['Address'].values:
  print(address)
  geocode_address(address)

# Add new columns to dataframe
df['Latitude'] = coord_lat
df['Longitude'] = coord_lon
display(df)

Now that we have a dataframe including coordinates, we can write the new data to GeoJSON.


In [ ]:
# Convert to GeoJSON
features = []
for _, row in df.iterrows():
    features.append({
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['Longitude'], row['Latitude']],
        },
        "properties": row.drop(['Latitude', 'Longitude']).to_dict(),
    })

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Save GeoJSON to file
geojson_filename = "geocoded_output.geojson"
with open(geojson_filename, 'w') as f:
    json.dump(geojson_data, f)

# Download file from Colab
files.download(geojson_filename)

The final step is to use ``` folium ``` for the creation of simple zoomable map.




In [ ]:
# Display map
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=6)
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Address']
    ).add_to(m)
m

Monika Barget, Maastricht University, last edited in May 2025 -- The 2023 version of this code depricated when problems with problems with `ipywidgets` and `ipyleaflet` occurred.